# Checking Location Usage Policy of a Container
for this example we will be using `rdflib` which can be installed using pip

`pip install rdflib`

Since the usage policy contains a named graph, we will use `rdflib.ConjunctiveGraph` class instead of `rdflib.Graph`.

In [1]:
from rdflib import ConjunctiveGraph, Namespace, RDF, RDFS, OWL, URIRef

In order to access Usage Policy of a semantic container, we access via API on `/api/meta/usage` which will return a usage policy in RDF graph (using `trig` serialization format)

For convenience in querying an RDF Graph, we define some Namespace which we will bind it to some prefixes in the graph later on.

In [17]:
#SEMCON_URL = "https://vownyourdata.zamg.ac.at:9600/api/meta/"
SEMCON_URL = "https://vownyourdata.zamg.ac.at:9505/api/meta/"

SC_FILE = Namespace(SEMCON_URL)
SCO = Namespace("http://w3id.org/semcon/ns/ontology#")
SPL = Namespace("http://www.specialprivacy.eu/langs/usage-policy#")
SPL_LOC = Namespace("http://www.specialprivacy.eu/vocabs/locations#")


In [18]:
g = ConjunctiveGraph()

#namespace binding, useful for printout and querying
g.bind('owl', OWL)
g.bind('', SC_FILE)
g.bind('sc', SCO)
g.bind('spl', SPL)
g.bind('loc', SPL_LOC)

#load the actual data
g.parse(SEMCON_URL+"usage", format="trig")

#print the number of RDF triples
print(len(g))

40


First thing first, we will have a look on the policy

In [19]:
# print some human-readable serialization
print(g.serialize(format="trig").decode("utf-8"))

@prefix : <https://vownyourdata.zamg.ac.at:9505/api/meta/> .
@prefix loc: <http://www.specialprivacy.eu/vocabs/locations#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix sc: <http://w3id.org/semcon/ns/ontology#> .
@prefix spl: <http://www.specialprivacy.eu/langs/usage-policy#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

sc:UsagePolicy {
    :ContainerPolicy a owl:Class ;
        owl:equivalentClass [ a owl:Class ;
                owl:intersectionOf ( [ a owl:Restriction ;
                            owl:onProperty spl:hasData ;
                            owl:someValuesFrom spl:AnyData ] [ a owl:Restriction ;
                            owl:onProperty spl:hasProcessing ;
                            owl:someValuesFrom spl:AnyProcessing ] [ a owl:Restriction ;
                            owl:onPrope

The policy is defined using OWL vocabulary as a set of restriction on each aspect : Data policy, Processing policy, purpose, recipient, and storage (which contain location and duration).

instead of 'pretty' format above, we can also see how it look as list of triples of subject, predicate, and object

In [26]:
# print list of triples
print(g.serialize(format="nt").decode("utf-8"))

_:n4 <http://www.w3.org/2002/07/owl#onProperty> <http://www.specialprivacy.eu/langs/usage-policy#hasPurpose> .
_:n6 <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://www.w3.org/2002/07/owl#Restriction> .
_:n6 <http://www.w3.org/2002/07/owl#onProperty> <http://www.specialprivacy.eu/langs/usage-policy#hasStorage> .
_:n12 <http://www.w3.org/1999/02/22-rdf-syntax-ns#first> _:n2 .
_:n3 <http://www.w3.org/2002/07/owl#onProperty> <http://www.specialprivacy.eu/langs/usage-policy#hasProcessing> .
_:n16 <http://www.w3.org/1999/02/22-rdf-syntax-ns#rest> <http://www.w3.org/1999/02/22-rdf-syntax-ns#nil> .
_:n5 <http://www.w3.org/2002/07/owl#someValuesFrom> <http://www.specialprivacy.eu/langs/usage-policy#AnyRecipient> .
<https://vownyourdata.zamg.ac.at:9505/api/meta/ContainerPolicy> <http://www.w3.org/2002/07/owl#equivalentClass> _:n1 .
_:n6 <http://www.w3.org/2002/07/owl#someValuesFrom> _:n7 .
_:n9 <http://www.w3.org/2002/07/owl#onProperty> <http://www.specialprivacy.eu/langs/usage-policy#

In order to extract information from the graph, we will use SPARQL query on the graph. 

In [23]:
qloc = """
SELECT ?q 
WHERE { 
?policy a owl:Class; owl:equivalentClass [
    owl:intersectionOf ?i
    ].
?i rdf:rest*/rdf:first [
    owl:onProperty spl:hasStorage;
    owl:someValuesFrom/owl:intersectionOf ?s
    ].
?s rdf:rest*/rdf:first [
    owl:onProperty spl:hasLocation;
    owl:someValuesFrom ?q
    ].
}"""
q = g.query(qloc)
for s in q:
    print(s)

(rdflib.term.URIRef('http://www.specialprivacy.eu/langs/usage-policy#AnyLocation'),)


In [25]:
qunion = """
SELECT ?q 
WHERE { 
?policy a owl:Class; owl:equivalentClass [
    owl:intersectionOf ?i
    ].
?i rdf:rest*/rdf:first [
    owl:onProperty spl:hasStorage;
    owl:someValuesFrom/owl:intersectionOf ?s
    ].
?s rdf:rest*/rdf:first [
    owl:onProperty spl:hasLocation;
    owl:someValuesFrom/owl:unionOf ?l
    ].
?l rdf:rest*/rdf:first ?q.
}
"""
r = list(q)
#check if the returned object is a union or not
if isinstance(r[0][0], URIRef):
    print(r[0][0])
else:
    loc = g.query(qunion)
    for l in loc:
        print(l[0])

http://www.specialprivacy.eu/langs/usage-policy#AnyLocation
